# Dropped Features 

- avg_energy_kWh
Ye ek derived aur inconsistent feature hai, jo hourly charging demand ko properly explain nahi karta.

- energy_consumption_kWh
Energy consumption charging sessions ke baad pata chalta hai, isliye ye target leakage cause karta hai.

- peak_power_kW
Peak power charging ke baad record hota hai aur energy ke saath highly correlated hota hai, isliye demand predict karne ke liye suitable nahi hai.

- idle_duration_minutes
Idle duration directly batata hai ki us hour me charging hui ya nahi, isliye model ko future ka answer pehle hi mil jata hai (data leakage).

- active_duration_minutes
Active duration bhi charging hone ke baad milta hai aur prediction time pe available nahi hota.

### 
humne post-event aur leakage-causing features ko hata diya taaki demand forecasting model honest aur realistic rahe

# Goal 
- Primary Goal (num_sessions) : num_sessions actual charging demand dikhata hai, aur backend load planning, demand heatmap aur system utilization sab isi par depend karta hai.

- Secondary Goal (Pricing) : Price ko directly predict nahi karte, balki forecasted demand ke basis par future price ko adjust ya suggest karte hain.

In [3]:
import pandas as pd
import numpy as np
DATA_PATH = r"F:\task\ev_charging_forecast\data\raw\karocharge_delhi_30day_realistic_synthetic_mixed_idle.csv"
data = pd.read_csv(DATA_PATH)

In [ ]:
data = data.copy()

data.drop(
    columns=[
        "avg_energy_kWh",
        "energy_consumption_kWh",
        "peak_power_kW",
        "idle_duration_minutes",
        "active_duration_minutes",
        "station_id",
        "is_idle_time",
        "price_inr"
    ],
    inplace=True
)
data.head()

,timestamp,num_sessions,is_holiday,weather_temp,weather_condition
0,09/11/2025 01:00,1,1,10.970434,Haze
1,09/11/2025 02:00,1,1,13.846141,Clear
2,09/11/2025 03:00,0,1,10.982564,Haze
3,09/11/2025 04:00,3,1,12.506471,Partly Cloudy
4,09/11/2025 05:00,0,1,12.345760,Clear


In [4]:
# make sure timestamp is datetime
data["timestamp"] = pd.to_datetime(
    data["timestamp"], format="%d/%m/%Y %H:%M"
)

# set timestamp as index
data.set_index("timestamp", inplace=True)

# now create time features
data["hour"] = data.index.hour
data["day_of_week"] = data.index.dayofweek
data["is_weekend"] = data["day_of_week"].isin([5, 6]).astype(int)

data.head()


,station_id,num_sessions,avg_energy_kWh,energy_consumption_kWh,peak_power_kW,price_inr,is_holiday,weather_temp,weather_condition,is_idle_time,idle_duration_minutes,active_duration_minutes,hour,day_of_week,is_weekend
timestamp,,,,,,,,,,,,,,,
2025-11-09 01:00:00,STN_DELHI_01,1,14.322362,9.548241,29.745500,16.532235,1,10.970434,Haze,0,20,40,1,6,1
2025-11-09 02:00:00,STN_DELHI_01,1,14.945375,14.945375,22.472012,15.626717,1,13.846141,Clear,0,0,60,2,6,1
2025-11-09 03:00:00,STN_DELHI_01,0,0.000000,0.000000,0.000000,13.753393,1,10.982564,Haze,1,45,15,3,6,1
2025-11-09 04:00:00,STN_DELHI_01,3,11.689493,23.378985,10.986084,18.761273,1,12.506471,Partly Cloudy,0,20,40,4,6,1
2025-11-09 05:00:00,STN_DELHI_01,0,0.000000,0.000000,0.000000,14.099645,1,12.345760,Clear,1,60,0,5,6,1


feature that will be helpful
- lagged demand (short lags)

In [5]:
# STEP 5.2: Short lag features

data["lag_1h"] = data["num_sessions"].shift(1)
data["lag_3h"] = data["num_sessions"].shift(3)
data["lag_5h"] = data["num_sessions"].shift(5)
data["lag_7h"] = data["num_sessions"].shift(7)

data.head(10)


,station_id,num_sessions,avg_energy_kWh,energy_consumption_kWh,peak_power_kW,price_inr,is_holiday,weather_temp,weather_condition,is_idle_time,idle_duration_minutes,active_duration_minutes,hour,day_of_week,is_weekend,lag_1h,lag_3h,lag_5h,lag_7h
timestamp,,,,,,,,,,,,,,,,,,,
2025-11-09 01:00:00,STN_DELHI_01,1,14.322362,9.548241,29.745500,16.532235,1,10.970434,Haze,0,20,40,1,6,1,NaN,NaN,NaN,NaN
2025-11-09 02:00:00,STN_DELHI_01,1,14.945375,14.945375,22.472012,15.626717,1,13.846141,Clear,0,0,60,2,6,1,1.0,NaN,NaN,NaN
2025-11-09 03:00:00,STN_DELHI_01,0,0.000000,0.000000,0.000000,13.753393,1,10.982564,Haze,1,45,15,3,6,1,1.0,NaN,NaN,NaN
2025-11-09 04:00:00,STN_DELHI_01,3,11.689493,23.378985,10.986084,18.761273,1,12.506471,Partly Cloudy,0,20,40,4,6,1,0.0,1.0,NaN,NaN
2025-11-09 05:00:00,STN_DELHI_01,0,0.000000,0.000000,0.000000,14.099645,1,12.345760,Clear,1,60,0,5,6,1,3.0,1.0,NaN,NaN
2025-11-09 06:00:00,STN_DELHI_01,0,0.000000,0.000000,0.000000,14.893517,1,12.710807,Clear,1,60,0,6,6,1,0.0,0.0,1.0,NaN
2025-11-09 07:00:00,STN_DELHI_01,0,0.000000,0.000000,0.000000,13.547827,1,17.315234,Haze,1,45,15,7,6,1,0.0,3.0,1.0,NaN
2025-11-09 08:00:00,STN_DELHI_01,1,13.378589,12.263707,27.014641,16.082042,1,18.837910,Haze,0,5,55,8,6,1,0.0,0.0,0.0,1.0
2025-11-09 09:00:00,STN_DELHI_01,0,0.000000,0.000000,0.000000,13.353364,1,21.603835,Clear,1,30,30,9,6,1,1.0,0.0,3.0,1.0


In [6]:
data["lag_1h"] = data["lag_1h"].fillna(0)
data["lag_3h"] = data["lag_3h"].fillna(0)
data["lag_5h"] = data["lag_5h"].fillna(0)
data["lag_7h"] = data["lag_7h"].fillna(0)

In [6]:
data

,num_sessions,is_holiday,weather_temp,weather_condition,hour,day_of_week,is_weekend,lag_1h,lag_3h,lag_5h,lag_7h
timestamp,,,,,,,,,,,
2025-11-09 01:00:00,1,1,10.970434,Haze,1,6,1,0.0,0.0,0.0,0.0
2025-11-09 02:00:00,1,1,13.846141,Clear,2,6,1,1.0,0.0,0.0,0.0
2025-11-09 03:00:00,0,1,10.982564,Haze,3,6,1,1.0,0.0,0.0,0.0
2025-11-09 04:00:00,3,1,12.506471,Partly Cloudy,4,6,1,0.0,1.0,0.0,0.0
2025-11-09 05:00:00,0,1,12.345760,Clear,5,6,1,3.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2025-12-08 21:00:00,2,0,14.929285,Partly Cloudy,21,0,0,2.0,1.0,2.0,0.0
2025-12-08 22:00:00,1,0,12.400268,Haze,22,0,0,2.0,0.0,1.0,0.0
2025-12-08 23:00:00,1,0,13.617752,Clear,23,0,0,1.0,2.0,1.0,2.0


In [7]:
# CELL 1: Check current data structure
print("=== DATA STRUCTURE CHECK ===")
print(f"Data type: {type(data)}")
print(f"Data shape: {data.shape}")
print(f"\nColumns present: {data.columns.tolist()}")
print(f"\nIndex type: {type(data.index)}")
print(f"Index name: {data.index.name}")
print(f"\nFirst 5 rows:")
print(data.head())
print(f"\nMissing values in each column:")
print(data.isnull().sum())

=== DATA STRUCTURE CHECK ===
Data type: <class 'pandas.DataFrame'>
Data shape: (721, 19)

Columns present: ['station_id', 'num_sessions', 'avg_energy_kWh', 'energy_consumption_kWh', 'peak_power_kW', 'price_inr', 'is_holiday', 'weather_temp', 'weather_condition', 'is_idle_time', 'idle_duration_minutes', 'active_duration_minutes', 'hour', 'day_of_week', 'is_weekend', 'lag_1h', 'lag_3h', 'lag_5h', 'lag_7h']

Index type: <class 'pandas.DatetimeIndex'>
Index name: timestamp

First 5 rows:
                       station_id  num_sessions  avg_energy_kWh  \
timestamp                                                         
2025-11-09 01:00:00  STN_DELHI_01             1       14.322362   
2025-11-09 02:00:00  STN_DELHI_01             1       14.945375   
2025-11-09 03:00:00  STN_DELHI_01             0        0.000000   
2025-11-09 04:00:00  STN_DELHI_01             3       11.689493   
2025-11-09 05:00:00  STN_DELHI_01             0        0.000000   

                     energy_consumption_k

In [8]:
# CELL 2: Add cyclical features
print("=== ADDING CYCLICAL FEATURES ===")

import numpy as np

# Add cyclical time features
data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)
data['day_sin'] = np.sin(2 * np.pi * data['day_of_week'] / 7)
data['day_cos'] = np.cos(2 * np.pi * data['day_of_week'] / 7)

# Add time of day categories
data['is_night'] = (data['hour'].between(0, 5)).astype(int)
data['is_morning'] = (data['hour'].between(6, 11)).astype(int)
data['is_afternoon'] = (data['hour'].between(12, 16)).astype(int)
data['is_evening'] = (data['hour'].between(17, 23)).astype(int)

print(f"✅ Added 8 new features")
print(f"Total columns now: {len(data.columns)}")
print(f"\nNew columns: {[col for col in data.columns if col not in ['num_sessions', 'is_holiday', 'weather_temp', 'weather_condition', 'hour', 'day_of_week', 'is_weekend', 'lag_1h', 'lag_3h', 'lag_5h', 'lag_7h']]}")
print(f"\nSample of new features:")
print(data[['hour', 'hour_sin', 'hour_cos', 'is_night', 'is_morning']].head())

=== ADDING CYCLICAL FEATURES ===
✅ Added 8 new features
Total columns now: 27

New columns: ['station_id', 'avg_energy_kWh', 'energy_consumption_kWh', 'peak_power_kW', 'price_inr', 'is_idle_time', 'idle_duration_minutes', 'active_duration_minutes', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'is_night', 'is_morning', 'is_afternoon', 'is_evening']

Sample of new features:
                     hour  hour_sin  hour_cos  is_night  is_morning
timestamp                                                          
2025-11-09 01:00:00     1  0.258819  0.965926         1           0
2025-11-09 02:00:00     2  0.500000  0.866025         1           0
2025-11-09 03:00:00     3  0.707107  0.707107         1           0
2025-11-09 04:00:00     4  0.866025  0.500000         1           0
2025-11-09 05:00:00     5  0.965926  0.258819         1           0


In [9]:
# CELL 3: Add rolling statistics
print("=== ADDING ROLLING STATISTICS ===")

# Add rolling features
# data['rolling_mean_3h'] = data['num_sessions'].rolling(window=3, min_periods=1).mean()
data['rolling_mean_6h'] = data['num_sessions'].rolling(window=6, min_periods=1).mean()
data['rolling_mean_12h'] = data['num_sessions'].rolling(window=12, min_periods=1).mean()
data['rolling_mean_24h'] = data['num_sessions'].rolling(window=24, min_periods=1).mean()

# Fill NaN values
rolling_cols = ['rolling_mean_6h', 'rolling_mean_12h', 'rolling_mean_24h']
for col in rolling_cols:
    data[col] = data[col].fillna(0)

print(f"✅ Added 4 rolling features")
print(f"Total columns now: {len(data.columns)}")
print(f"\nRolling features stats:")
print(data[rolling_cols].describe())
print(f"\nSample rolling values:")
print(data[['num_sessions', 'rolling_mean_6h']].head(10))

=== ADDING ROLLING STATISTICS ===
✅ Added 4 rolling features
Total columns now: 30

Rolling features stats:
       rolling_mean_6h  rolling_mean_12h  rolling_mean_24h
count       721.000000        721.000000        721.000000
mean          0.847781          0.846727          0.846415
std           0.385943          0.254451          0.159513
min           0.000000          0.166667          0.375000
25%           0.500000          0.666667          0.708333
50%           0.833333          0.833333          0.875000
75%           1.000000          1.000000          0.958333
max           2.000000          1.666667          1.333333

Sample rolling values:
                     num_sessions  rolling_mean_6h
timestamp                                         
2025-11-09 01:00:00             1         1.000000
2025-11-09 02:00:00             1         1.000000
2025-11-09 03:00:00             0         0.666667
2025-11-09 04:00:00             3         1.250000
2025-11-09 05:00:00            

In [11]:
df.columns

Index(['num_sessions', 'is_holiday', 'weather_temp', 'weather_condition',
       'hour', 'day_of_week', 'is_weekend', 'lag_1h', 'lag_3h', 'lag_5h',
       'lag_7h', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'is_night',
       'is_morning', 'is_afternoon', 'is_evening', 'rolling_mean_6h',
       'rolling_mean_12h', 'rolling_mean_24h'],
      dtype='str')

In [10]:
OUTPUT_PATH = "../data/processed/ev_charging_analysis_ready1.csv"

df.to_csv(OUTPUT_PATH, index=True)

print(f"✅ Data saved successfully at: {OUTPUT_PATH}")

✅ Data saved successfully at: ../data/processed/ev_charging_analysis_ready1.csv
